In [1]:
#working code
import pandas as pd

# Load data
data = pd.read_excel('PPKGT_W_ONLY_CLMTYPE_P_ALR.xlsx')

c:\Users\FMohiuddin\OneDrive - AArete, LLC\Desktop\CenteneXcelys\mynev\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [2]:
import pandas as pd

# Group by PPKGT_TEMPLATE_ID
grouped = data.groupby('PPKGT_TEMPLATE_ID')
print("length after ID grouping", len(grouped))
specified_price_rules = ['DMETR', 'FFSTR', 'MCRIL']

# Function to check if any price rule in a group is outside of the specified rules
def has_invalid_rule(group):
    unique_rules = group['PRICE_RULE_1'].unique()
    return any(rule not in specified_price_rules for rule in unique_rules)

# Find valid PPKGT_TEMPLATE_IDs (Groups without invalid rules and with more than one item)
valid_template_ids = [name for name, group in grouped if not has_invalid_rule(group) and len(group) >= 1]
print("length after removing invalid price rules", len(valid_template_ids))

# Filter out the valid groups
final_filtered_data = data[data['PPKGT_TEMPLATE_ID'].isin(valid_template_ids)]

# Print the sizes of the final groups
group_size_counts = final_filtered_data.groupby('PPKGT_TEMPLATE_ID').size().value_counts()
print()
filtered_group_size_counts = group_size_counts[group_size_counts > 1]
print(filtered_group_size_counts)
print(len(group_size_counts))


length after ID grouping 626
length after removing invalid price rules 348

23       121
1         15
19        11
51         9
43         8
131        8
848        8
4687       8
54         8
57         7
60         6
3499       6
2          6
127        5
4          4
7          4
116        4
82         3
12         3
35         3
143        3
110        3
3110       3
81         3
5          2
120        2
32         2
964        2
91         2
10         2
17915      2
84         2
85         2
4923       2
4559       2
7724       2
2934       2
134        2
4395       2
48         2
908        2
53         2
Name: count, dtype: int64
95


In [3]:
cols_to_compare = [
    'PRICE_SCHEDULE', 'SCHED_PCT_ALLOWED', 'PCT_ALLOWED', 
    'PCT_OF_BILLED', 'PRICE_RULE_1', 'DETERMINANT', 'DETERMINANT_TABLE',
    'PCT_WITHHOLD'
]

duplicates = []

# Get group sizes that appear more than once
valid_group_sizes = filtered_group_size_counts.index.tolist()

for size in valid_group_sizes:
    # Get all groups of the current size
    groups_of_size = final_filtered_data.groupby('PPKGT_TEMPLATE_ID').filter(lambda x: len(x) == size)
    
    unique_ppkgts = groups_of_size['PPKGT_TEMPLATE_ID'].unique()

    for i in range(len(unique_ppkgts)):
        for j in range(i+1, len(unique_ppkgts)):
            group1 = groups_of_size[groups_of_size['PPKGT_TEMPLATE_ID'] == unique_ppkgts[i]]
            group2 = groups_of_size[groups_of_size['PPKGT_TEMPLATE_ID'] == unique_ppkgts[j]]
            
            matched = True
            for col in cols_to_compare:
                if col == 'PRICE_RULE_1':
                    valid_price_rules = ['DMETR', 'FFSTR', 'MCRIL']
                    if not (group1[col].iloc[0] in valid_price_rules and group2[col].iloc[0] in valid_price_rules):
                        print(f"Invalid price rule compared: {group1[col].iloc[0]} and {group2[col].iloc[0]}")
                
                if not group1[col].reset_index(drop=True).equals(group2[col].reset_index(drop=True)):
                    matched = False
                    break
            
            if matched:
                # Further check for 'DET_VALUE_FROM' and 'DET_VALUE_TO' columns
                
                if  sorted(group1['DET_VALUE_FROM'].astype(str).unique()) == sorted(group2['DET_VALUE_FROM'].astype(str).unique()) and \
                    sorted(group1['DET_VALUE_TO'].astype(str).unique()) == sorted(group2['DET_VALUE_TO'].astype(str).unique()):
                
                    duplicates.append((unique_ppkgts[i], unique_ppkgts[j]))
                
# Print duplicates
for dup in duplicates:
    print(f"Duplicates found: PPKGT_TEMPLATE_ID {dup[0]} and {dup[1]}")
print(len(duplicates))

# create a dataframe with both dup[0] and dup[1] and then export to excel
df = pd.DataFrame(duplicates)
df.to_excel("duplicates.xlsx")


Duplicates found: PPKGT_TEMPLATE_ID ABMPALS100 and ALAPSP100
Duplicates found: PPKGT_TEMPLATE_ID ABMPALS100 and ALPPPPC100
Duplicates found: PPKGT_TEMPLATE_ID ABMPALS100 and ALPPPSP100
Duplicates found: PPKGT_TEMPLATE_ID ABMPPC03 and ABMPSP03
Duplicates found: PPKGT_TEMPLATE_ID ABMPPC04 and ABMPSP003
Duplicates found: PPKGT_TEMPLATE_ID ABMPPC04 and ABMPSP05
Duplicates found: PPKGT_TEMPLATE_ID ABMPPC05 and ABMPSP06
Duplicates found: PPKGT_TEMPLATE_ID ABMPSP003 and ABMPSP05
Duplicates found: PPKGT_TEMPLATE_ID ALAPSP100 and ALPPPPC100
Duplicates found: PPKGT_TEMPLATE_ID ALAPSP100 and ALPPPSP100
Duplicates found: PPKGT_TEMPLATE_ID ALPPPAL301 and ALRPAL100
Duplicates found: PPKGT_TEMPLATE_ID ALPPPAL301 and ALRPAL14
Duplicates found: PPKGT_TEMPLATE_ID ALPPPPC100 and ALPPPSP100
Duplicates found: PPKGT_TEMPLATE_ID ALRPAL100 and ALRPAL14
Duplicates found: PPKGT_TEMPLATE_ID ARKPPC103 and ARKPSP103
Duplicates found: PPKGT_TEMPLATE_ID G01PSP431 and G01PSP434
Duplicates found: PPKGT_TEMPLATE_ID MPP

In [34]:
def lists_are_equal(list1, list2):
    # Compare sorted lists
    return sorted(list1) == sorted(list2)

def nan_safe_compare(col1, col2):
    # Convert columns to string, treating NaNs as the same
    str_col1 = col1.astype(str).replace('nan', '').unique()
    str_col2 = col2.astype(str).replace('nan', '').unique()

    # Return if lists are equal
    return lists_are_equal(str_col1, str_col2)

In [39]:
#working LAST code
# Columns to compare for matching


cols_to_compare = [
    'PRICE_SCHEDULE', 'SCHED_PCT_ALLOWED', 'PCT_ALLOWED', 
    'PCT_OF_BILLED', 'PRICE_RULE_1', 'DETERMINANT', 'DETERMINANT_TABLE',
    'PCT_WITHHOLD'
]

duplicates = []

# Get group sizes that appear more than once
valid_group_sizes = filtered_group_size_counts.index.tolist()

for size in valid_group_sizes:
    # Get all groups of the current size
    groups_of_size = final_filtered_data.groupby('PPKGT_TEMPLATE_ID').filter(lambda x: len(x) == size)
    
    unique_ppkgts = groups_of_size['PPKGT_TEMPLATE_ID'].unique()

    for i in range(len(unique_ppkgts)):
        for j in range(i+1, len(unique_ppkgts)):
            group1 = groups_of_size[groups_of_size['PPKGT_TEMPLATE_ID'] == unique_ppkgts[i]]
            group2 = groups_of_size[groups_of_size['PPKGT_TEMPLATE_ID'] == unique_ppkgts[j]]
            
            matched = True
            for col in cols_to_compare:
                # Use .equals() method instead of the == operator
                if not group1[col].reset_index(drop=True).equals(group2[col].reset_index(drop=True)):
                    #print if price rule is not found in  in ['DMETR', 'FFSTR', 'MCRIL']:
                    if col == 'PRICE_RULE_1':
                        print(f"price rule is {group1[col].reset_index(drop=True)} and {group2[col].reset_index(drop=True)}")
                    matched = False
                    break
            
            if matched:
                # Further check for 'DET_VALUE_FROM' and 'DET_VALUE_TO' columns
                
                # Check if the 'DET_VALUE_FROM' and 'DET_VALUE_TO' columns are the same 
                if nan_safe_compare(group1['DET_VALUE_FROM'], group2['DET_VALUE_FROM']) and \
                nan_safe_compare(group1['DET_VALUE_TO'], group2['DET_VALUE_TO']):
                    duplicates.append((unique_ppkgts[i], unique_ppkgts[j]))

# Print duplicates
#for dup in duplicates:
    
#    print(f"Duplicates found: PPKGT_TEMPLATE_ID {dup[0]} and {dup[1]}")
#print(len(duplicates))

#create a dataframe with both dup[0] and dup[1] and then export to excel
df = pd.DataFrame(duplicates)
df.to_excel("duplicates.xlsx")


In [35]:
# Consider only the first group size
size = valid_group_sizes[0]

# Get all groups of the current size
groups_of_size = final_filtered_data.groupby('PPKGT_TEMPLATE_ID').filter(lambda x: len(x) == size)
    
unique_ppkgts = groups_of_size['PPKGT_TEMPLATE_ID'].unique()

# For simplicity, we'll consider only the first PPKGT_TEMPLATE_ID for comparison
group1 = groups_of_size[groups_of_size['PPKGT_TEMPLATE_ID'] == unique_ppkgts[0]]

for j in range(1, len(unique_ppkgts)):
    group2 = groups_of_size[groups_of_size['PPKGT_TEMPLATE_ID'] == unique_ppkgts[j]]
    
    matched_columns_count = 0
    for col in cols_to_compare:
        
        if (group1[col].reset_index(drop=True) == group2[col].reset_index(drop=True)).all():
            print(col)
            matched_columns_count += 1
        else:
            print(f"Columns {col} do not match for PPKGT_TEMPLATE_ID {unique_ppkgts[0]} and {unique_ppkgts[j]}")
            
    print(f"{matched_columns_count}/{len(cols_to_compare)} columns matched for PPKGT_TEMPLATE_ID {unique_ppkgts[0]} and {unique_ppkgts[j]}")

    if matched_columns_count == len(cols_to_compare):
        # Further check for 'DET_VALUE_FROM' and 'DET_VALUE_TO' columns
        if sorted(group1['DET_VALUE_FROM'].unique()) == sorted(group2['DET_VALUE_FROM'].unique()) and \
           sorted(group1['DET_VALUE_TO'].unique()) == sorted(group2['DET_VALUE_TO'].unique()):
            print(f"Duplicates found: PPKGT_TEMPLATE_ID {unique_ppkgts[0]} and {unique_ppkgts[j]}")
        else:
            print(f"DET_VALUE columns do not match for PPKGT_TEMPLATE_ID {unique_ppkgts[0]} and {unique_ppkgts[j]}")


PRICE_SCHEDULE
SCHED_PCT_ALLOWED
Columns PCT_ALLOWED do not match for PPKGT_TEMPLATE_ID ARKPALSP12 and ARKPALSP14
Columns PCT_OF_BILLED do not match for PPKGT_TEMPLATE_ID ARKPALSP12 and ARKPALSP14
PRICE_RULE_1
Columns DETERMINANT do not match for PPKGT_TEMPLATE_ID ARKPALSP12 and ARKPALSP14
Columns DETERMINANT_TABLE do not match for PPKGT_TEMPLATE_ID ARKPALSP12 and ARKPALSP14
Columns PCT_WITHHOLD do not match for PPKGT_TEMPLATE_ID ARKPALSP12 and ARKPALSP14
3/8 columns matched for PPKGT_TEMPLATE_ID ARKPALSP12 and ARKPALSP14
PRICE_SCHEDULE
Columns SCHED_PCT_ALLOWED do not match for PPKGT_TEMPLATE_ID ARKPALSP12 and ARKPBH1171
Columns PCT_ALLOWED do not match for PPKGT_TEMPLATE_ID ARKPALSP12 and ARKPBH1171
Columns PCT_OF_BILLED do not match for PPKGT_TEMPLATE_ID ARKPALSP12 and ARKPBH1171
PRICE_RULE_1
Columns DETERMINANT do not match for PPKGT_TEMPLATE_ID ARKPALSP12 and ARKPBH1171
Columns DETERMINANT_TABLE do not match for PPKGT_TEMPLATE_ID ARKPALSP12 and ARKPBH1171
Columns PCT_WITHHOLD do n

In [39]:
print("Comparing columns for PPKGT_TEMPLATE_ID 'M01PALSP1' and 'M01ALSP100':\n")
for col in cols_to_compare:
    if group1[col].reset_index(drop=True).equals(group2[col].reset_index(drop=True)):
        print(f"Column '{col}' MATCHES for both PPKGT_TEMPLATE_IDs")
    else:
        print(f"Column '{col}' DOES NOT match for both PPKGT_TEMPLATE_IDs")
        print("Values in 'M01PALSP1':")
        print(group1[col].reset_index(drop=True).to_string())
        print("\nValues in 'M01ALSP100':")
        print(group2[col].reset_index(drop=True).to_string())
        print("------------------------------")


Comparing columns for PPKGT_TEMPLATE_ID 'M01PALSP1' and 'M01ALSP100':

Column 'PRICE_SCHEDULE' MATCHES for both PPKGT_TEMPLATE_IDs
Column 'SCHED_PCT_ALLOWED' MATCHES for both PPKGT_TEMPLATE_IDs
Column 'PCT_ALLOWED' MATCHES for both PPKGT_TEMPLATE_IDs
Column 'PCT_OF_BILLED' MATCHES for both PPKGT_TEMPLATE_IDs
Column 'PRICE_RULE_1' MATCHES for both PPKGT_TEMPLATE_IDs
Column 'DETERMINANT' MATCHES for both PPKGT_TEMPLATE_IDs
Column 'DETERMINANT_TABLE' MATCHES for both PPKGT_TEMPLATE_IDs
Column 'PCT_WITHHOLD' MATCHES for both PPKGT_TEMPLATE_IDs


In [4]:
import pandas as pd
# Load data
data = pd.read_excel('PPKGT_W_ONLY_CLMTYPE_P_ALR.xlsx')
# Group by PPKGT_TEMPLATE_ID
grouped = data.groupby('PPKGT_TEMPLATE_ID')
print("length after ID grouping", len(grouped))
specified_price_rules = ['DMETR', 'FFSTR', 'MCRIL']

# Function to check if any price rule in a group is outside of the specified rules
def has_invalid_rule(group):
    unique_rules = group['PRICE_RULE_1'].unique()
    return any(rule not in specified_price_rules for rule in unique_rules)

# Find valid PPKGT_TEMPLATE_IDs (Groups without invalid rules and with more than one item)
valid_template_ids = [name for name, group in grouped if not has_invalid_rule(group) and len(group) >= 1]
print("length after removing invalid price rules", len(valid_template_ids))

# Filter out the valid groups
final_filtered_data = data[data['PPKGT_TEMPLATE_ID'].isin(valid_template_ids)]

# Print the sizes of the final groups
group_size_counts = final_filtered_data.groupby('PPKGT_TEMPLATE_ID').size().value_counts()
print()
filtered_group_size_counts = group_size_counts[group_size_counts > 1]
print(filtered_group_size_counts)
print(len(group_size_counts))
cols_to_compare = [
    'PRICE_SCHEDULE', 'SCHED_PCT_ALLOWED', 'PCT_ALLOWED', 
    'PCT_OF_BILLED', 'PRICE_RULE_1', 'DETERMINANT', 'DETERMINANT_TABLE',
    'PCT_WITHHOLD'
]

duplicates = []

# Get group sizes that appear more than once
valid_group_sizes = filtered_group_size_counts.index.tolist()

for size in valid_group_sizes:
    # Get all groups of the current size
    groups_of_size = final_filtered_data.groupby('PPKGT_TEMPLATE_ID').filter(lambda x: len(x) == size)
    
    unique_ppkgts = groups_of_size['PPKGT_TEMPLATE_ID'].unique()

    for i in range(len(unique_ppkgts)):
        for j in range(i+1, len(unique_ppkgts)):
            group1 = groups_of_size[groups_of_size['PPKGT_TEMPLATE_ID'] == unique_ppkgts[i]]
            group2 = groups_of_size[groups_of_size['PPKGT_TEMPLATE_ID'] == unique_ppkgts[j]]
            
            matched = True
            for col in cols_to_compare:
                if col == 'PRICE_RULE_1':
                    valid_price_rules = ['DMETR', 'FFSTR', 'MCRIL']
                    if not (group1[col].iloc[0] in valid_price_rules and group2[col].iloc[0] in valid_price_rules):
                        print(f"Invalid price rule compared: {group1[col].iloc[0]} and {group2[col].iloc[0]}")
                
                if not group1[col].reset_index(drop=True).equals(group2[col].reset_index(drop=True)):
                    matched = False
                    break
            
            if matched:
                # Further check for 'DET_VALUE_FROM' and 'DET_VALUE_TO' columns
                
                if  sorted(group1['DET_VALUE_FROM'].astype(str).unique()) == sorted(group2['DET_VALUE_FROM'].astype(str).unique()) and \
                    sorted(group1['DET_VALUE_TO'].astype(str).unique()) == sorted(group2['DET_VALUE_TO'].astype(str).unique()):
                
                    duplicates.append((unique_ppkgts[i], unique_ppkgts[j]))
                
# Print duplicates
for dup in duplicates:
    print(f"Duplicates found: PPKGT_TEMPLATE_ID {dup[0]} and {dup[1]}")
print(len(duplicates))

# create a dataframe with both dup[0] and dup[1] and then export to excel
df = pd.DataFrame(duplicates)
df.to_excel("duplicates_PPKGT_W_ONLY_CLMTYPE_P_ALR.xlsx")


c:\Users\FMohiuddin\OneDrive - AArete, LLC\Desktop\CenteneXcelys\mynev\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


length after ID grouping 721
length after removing invalid price rules 372

23       110
1         20
51        16
85        12
60        11
81         8
4687       8
87         7
57         7
19         7
131        6
3499       6
116        6
1394       5
54         5
82         4
65         4
91         4
43         4
5          4
5692       3
18953      3
11         3
1376       3
10         3
63         3
119        2
7          2
4395       2
2934       2
4405       2
123        2
84         2
53         2
70         2
125        2
2          2
48         2
88         2
86         2
24         2
112        2
5698       2
75         2
7599       2
20         2
Name: count, dtype: int64
106
Duplicates found: PPKGT_TEMPLATE_ID ARKPAL1 and ARKPAL105
Duplicates found: PPKGT_TEMPLATE_ID ARKPAL100 and ARKPAL3
Duplicates found: PPKGT_TEMPLATE_ID ARKPAL100 and ARKPSP73
Duplicates found: PPKGT_TEMPLATE_ID ARKPAL100 and VBCARKAL2
Duplicates found: PPKGT_TEMPLATE_ID ARKPAL3 and ARKPSP73
Dupl